In [1]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import time

In [2]:
def readData(file):
	"""
	reads the ground truth file
	returns a 2D array with each
	row as GT pose(arranged row major form)
	array size should be 1101*12
	"""
	data = []
	for i in range(77):
		line = file.readline()
		line = line.split()
		arr = np.array(line)
		arr = arr.reshape(3, 4)
		data.append(arr)
	return data

In [3]:
def readPointCloud(filename):
	"""
	reads bin file and returns
	as m*4 np array
	all points are in meters
	you can filter out points beyond(in x y plane)
	50m for ease of computation
	and above or below 10m
	"""
	pcl = np.fromfile(filename, dtype=np.float32,count=-1)
	pcl = pcl.reshape([-1,4])
	return pcl

In [4]:
finalPointCloud=[]
pointsInWorldFrame=[]
txtfile=open("01.txt","r")
transforms = readData(txtfile)
txtfile.close()   

In [5]:
loc = "./01/0000" + str(10) + ".bin"
ar = readPointCloud(loc)
arr = np.array(ar[:,:3])
color = np.array(arr[:,:-1])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(arr)
tr = np.append(transforms[10],[[0,0,0,1]],axis=0)
o3d.geometry.PointCloud.transform(pcd,tr)
pcdf = pcd
for i in range(11,87):
    loc = "./01/0000" + str(i) + ".bin"
    ar = readPointCloud(loc)
    arr = np.array(ar[:,:3])
    color = np.array(arr[:,:-1])
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(arr)
    R = pcd.get_rotation_matrix_from_zyx((0,-np.pi/2,np.pi/2))
    pcd.rotate(R, (0,0,0))
    tr = np.append(transforms[i-11],[[0,0,0,1]],axis=0)
    o3d.geometry.PointCloud.transform(pcd,tr)
    pcdf += pcd
    

In [46]:
o3d.visualization.draw_geometries([pcdf])
o3d.io.write_point_cloud("./pcd.ply",pcd)

In [39]:
# def convertToWorldFrame(i, points):
#     print(points.shape)
#     camToLiDAR = np.array([[0, 0, 1],[-1, 0, 0],[0, -1, 0]])
#     LiDarTocam = np.array([[0,0,1],[0,0,-1],[0,-1,0]])
#     for num, p in enumerate(points):
#         #()transforms[i - 10].astype('float32')
#         #()p.astype('float32')
#         #()print(type(transforms[i - 10]))
#         #()print(transforms[i - 10].shape)
#         #()print(transforms[i - 10])
#         #()print(type(p))
#         print(p[0:3].reshape((3,1)).shape,LiDarTocam.shape)
#         py = p[0:3].reshape((3,1))
#         #from right to left, point from lidar is acted upon by the world pose whcih is then multiplied by the transform to get into the lidars frame
#         #()print(p)
#         #pointsInWorldFrame.append(np.matmul(float(transforms[i - 10]), float(p)))
#         #()transforms[i - 10] = np.matmul(camToLiDAR, transforms[i - 10])
#         #()exit(0)
        
# #         pointsInWorldFrame.append(np.matmul(camToLiDAR,np.matmul(transforms[i - 10].astype('float32'), p.astype('float32'))))
#         pointsInWorldFrame.append(np.matmul(np.matmul(py,LiDarTocam),transforms[i - 10].astype('float32')))
#     return pointsInWorldFrame
#     #()pointsInWorldFrame = np.array(pointsInWorldFrame)
#     #()print(pointsInWorldFrame.shape)
#     #()return pointsInWorldFrame



In [41]:
# #()

# if __name__ == "__main__":
#     finalPointCloud=[]
#     pointsInWorldFrame=[]
#     txtfile=open("01.txt","r")
#     transforms = readData(txtfile)
#     txtfile.close()
#     for i in range(10,87):
#         pf = []
#         loc = "./01/0000" + str(i) + ".bin"
#         output = readPointCloud(loc)
#         #()finalPointCloud.append(convertToWorldFrame(i, output))
#         pf = convertToWorldFrame(i, output,pointsInWorldFrame)
#         pointsInWorldFrame.append(pf)
#         #()print(type(output))
#         #()print(output.shape)
#         #()print(output)
#         #()for t in transforms:
#         #()		print(t)
#     #()	print(type(pointsInWorldFrame))
#     pointsInWorldFrame = np.array(pointsInWorldFrame)
#     print(pointsInWorldFrame.shape)
#     #()	print(pointsInWorldFrame[0])
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(pointsInWorldFrame)
#     o3d.visualization.draw_geometries([pcd])
#     o3d.io.write_point_cloud("./pcd1.ply",pcd)

TypeError: convertToWorldFrame() takes 2 positional arguments but 3 were given